# Customise scenarios in NZUpy

## Overview

This notebook demonstrates how to create and customise multiple scenarios in NZUpy. We'll explore:

1. Customising inputs with our own inputs not featured as an available 'config' in the model's pre-prepared CSVs
2. Generating comparative charts
3. Using our own custom functions to edit input parameters
4. The impact of some important model variables that are important to interpretation of results

## A1. Set Up the NZUpy Model

First, let's import the necessary libraries and initialise the model.

In [1]:
# Import necessary libraries
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from pathlib import Path
import sys
import os

# Import the NZUpy class
project_root = Path().absolute().parent
sys.path.insert(0, str(project_root))
from model.core.base_model import NZUpy

# Set our input and output directories
data_dir = project_root / "data"
output_dir = project_root / "examples" / "outputs" / "02_customise_scenarios"
os.makedirs(output_dir, exist_ok=True)

# Initialise NZUpy
NZU = NZUpy(data_dir=data_dir)

## A2. Define the scope

For this run, we'll look at an important parameter that can affect price outcomes - liquidity factor. This variable sets a limit in any given year on how many non-surplus stockpile units can be used to cover short-falls in supply. 

'Non-surplus' units are used by the Commission and Government to refer to units that have a clear/valid purpose for being held in private accounts. A notable example are units held by production foresters to cover their harvest liabilities in future.

In [ ]:
# Define time periods: start year, end year
NZU.define_time(2024, 2050)

# Define multiple scenarios for different liquidity factors & prime the model
NZU.define_scenarios(['Low Liquidity', 'Medium Liquidity', 'High Liquidity'])
NZU.prime()

## A3. Configure Scenarios with Different Liquidity Factors

There is little public reseach establishing what proportion of non-surplus units held in private accounts are readily liquid (tradeable) in any given year, so we'll run three scenarios exploring the impact of varying levels of liquidity, with custom liquidity factors of 0.05, 0.10, and 0.15 so that we can better understand the importance of this uncertain variable.

In [ ]:
# Start with central configs for all scenarios to load in sensible defaults for all inputs
NZU.use_central_configs(0)  # Low Liquidity scenario
NZU.use_central_configs(1)  # Medium Liquidity scenario 
NZU.use_central_configs(2)  # High Liquidity scenario

# Customise liquidity factors
NZU.set_parameter('liquidity_factor', 0.05, component='stockpile', scenario_index=0)
NZU.set_parameter('liquidity_factor', 0.15, component='stockpile', scenario_index=1)
NZU.set_parameter('liquidity_factor', 0.25, component='stockpile', scenario_index=2)

## A4. Run the Model and Generate Comparison Charts

Let's run the model with our different liquidity scenarios and visualise the results.

In [4]:
# Run the model
results = NZU.run()

NZUpy comes with a handy chart generator 

In [ ]:
# Load chart generator
from model.utils.chart_generator import ChartGenerator

# Initialise chart generator
chart_gen = ChartGenerator(NZU)


# Generate carbon price comparison chart for each scenario
for scenario_name in ['Low Liquidity', 'Medium Liquidity', 'High Liquidity']:
    price_chart = chart_gen.carbon_price_chart(scenario=scenario_name)
    display(price_chart)
    # Save each chart with a distinctive name
    price_chart.write_image(str(output_dir / f"liquidity_price_{scenario_name.lower().replace(' ', '_')}.png"))


## A5. Discussion of Results

The charts above reveals the importance of liquidity factor assumptions, with the change of a single variable seeing prices in 2050 varying from $133 to $214 (2023 real NZD). Stricter liquidty factor limits can see situations in which the carbon price ramps up to generate sufficient gross emissions mitigation to cover shortfalls in supply in the 2020s and early 2030s, highlighting the importance of this timeframe.

We can see this shortfall more specifically, and the volumes of non-surplus units used in the late 2020s by drawing on the supply components chart. We'll do this for the most and least strict liquidty scenarios of our three, and then print all three scenarios to a CSV for later analysis. 


In [ ]:
# First, let's show the supply components chart for the Low Liquidity scenario
supply_chart_strict = chart_gen.supply_components_chart(scenario='Low Liquidity')
display(supply_chart_strict)
supply_chart_strict.write_image(str(output_dir / "supply_components_lower_liquidity.png"))

# Then let's show the supply components chart for the High Liquidity scenario
supply_chart_loose = chart_gen.supply_components_chart(scenario='High Liquidity')
display(supply_chart_loose)
supply_chart_loose.write_image(str(output_dir / "supply_components_higher_liquidity.png"))

# Now let's export the data for all scenarios to CSV for further analysis
# We'll use the export_combined_data method which includes supply components
chart_gen.export_combined_data(str(output_dir / "liquidity_scenarios_comparison.csv"))

Here we can see a shortfall in supply relative to demand in the late 2020s for our low liquidty scenario, where we've restricted use of non-surplus units to just 5% of units held in private accounts. 

In our more scenario allowing more flexible use of non-surplus units however, the model is able to find sufficient supply without ramping up the price path to avoid an inbalance in supply and demand.



## B1. Modelling a Scarcity Scenario

Now, let's create a new NZUpy instance and set-up scenarios exploring temporary market scarcity in the mid/late-2020s further.

In [ ]:
# Reinitialise the model
NZU = NZUpy(data_dir=data_dir)
NZU.define_time(2024, 2050)

# Define two scenarios: baseline and scarcity scenario
NZU.define_scenarios(['Baseline', 'Scarcity then Surplus'])

# Prime the model
NZU.prime()

# Configure scenarios
NZU.use_central_configs(0)  # Set all components to central for baseline scenario
NZU.use_central_configs(1)  # Start with central configs for scarcity scenario

First we'll modify the second scenario to create scarcity by reducing auction volumes in 2025 so that no units are available (simulating situation in which no units clear).

In [ ]:
# Get and modify scarcity scenario's auction data
scarcity_data = NZU.show_inputs('auction', scenario_name='Scarcity then Surplus')
scarcity_volumes = scarcity_data['base_volume'].copy()
scarcity_volumes.loc[2025] = 0.0  # Keep it at zero for scarcity

# Update scarcity scenario
NZU.set_series('base_volume', scarcity_volumes, 'auction', scenario_name='Scarcity then Surplus')


In [ ]:
# Print volumes to verify they're different
print("Base Auction Volumes (2024-2029):")
print("\nBaseline Scenario:")
baseline_data = NZU.show_inputs('auction', scenario_name='Baseline')
for year in range(2024, 2030):
    print(f"    {year}: {baseline_data['base_volume'].loc[year]:,.1f} kt CO₂-e")



Great, we've constrained auction supply a bit, now let's adjust a recently incorporated feature of the Government's excel model - the price control parameter.

This parameter, sets a specific inflator/deflator to price annual price trends. While the default 'central' config sees the model's full (=1) price signal flow through each year, we can switch to a `scarcity_then_surplus` config in the base CSV file in `data/inputs/price/price_control.csv`, which forces a flip in price trends about 2030.

This exogenous switch attempts to mirror a situation in which increasing scarcity exists in the 2020s, followed by increasing supply (exceeding demand) in the 2030s. Users should take care with interpretation of results when playing around with the price control variable, as results are somewhat driven by assumptions made.

In [10]:
# Load the "scarcity_then_surplus" price control configuration only for the second scenario
NZU.use_price_control_config('scarcity_then_surplus', scenario_index=1)

# Run the model with our scenarios
results = NZU.run()

## B2. Visualising Scarcity Scenario Results

Let's create charts to visualise how our scarcity scenario affects prices and market dynamics.

In [ ]:
# Initialize chart generator
chart_gen = ChartGenerator(NZU)
# Generate and display carbon price chart for each scenario
print("Baseline Scenario Carbon Price:")
baseline_chart = chart_gen.carbon_price_chart(scenario='Baseline')
display(baseline_chart)

print("\nScarcity then Surplus Scenario Carbon Price:")
scarcity_chart = chart_gen.carbon_price_chart(scenario='Scarcity then Surplus')
display(scarcity_chart)


And we can also see the effect of these changes in input parameters on auction volumes cleared and auction revenues, where 

In [ ]:
# Generate and display auction revenue charts for each scenario
print("Baseline Scenario Auction Revenue:")
baseline_revenue_chart = chart_gen.auction_volume_revenue_chart(scenario='Baseline')
display(baseline_revenue_chart)

print("\nScarcity then Surplus Scenario Auction Revenue:")
scarcity_revenue_chart = chart_gen.auction_volume_revenue_chart(scenario='Scarcity then Surplus')
display(scarcity_revenue_chart)

# Save the charts
baseline_revenue_chart.write_image(str(output_dir / "baseline_auction_revenue.png"))
scarcity_revenue_chart.write_image(str(output_dir / "scarcity_auction_revenue.png"))

And we'll also use NZUpy's helper functions to generate an interactive html page with both charts, saved to `examples/outputs/02_customise_scenarios/price_comparison.html`

In [ ]:
# Create comparison page for specific chart type
from model.interface.chart_display import create_comparison_page

# Dictionary mapping scenario names to model instances
models = {
    'Baseline': (NZU, 'Baseline'),
    'Scarcity Then Surplus': (NZU, 'Scarcity then Surplus')
}

create_comparison_page(
    models=models,
    chart_type='carbon_price',
    output_dir=output_dir,
    filename="price_comparison.html",
    use_scenario_names=True
)

## B3. Discussion of Scarcity Scenario Results

The scarcity scenario demonstrates the importance of careful configuration and interpretation of scenarios. The **price spike** seen in the **scarcity_then_surplus** scenario results partly from the removal of auction volumes in 2025, together with the adjustment of the price control parameter amplifying this effect. 

Users should be careful when using the price control parameter in scenario runs, as carbon price trend results will tend to be a product of whatever assumptions are made, and hence should be well reasoned and justified.

## C1. Conclusion

This notebook has demonstrated:

1. **Creating multiple custom scenarios** with different parameter values.
2. **Modifying specific parameters** like the liquidity factor to values not available in predefined CSVs.
3. **Manually editing input series** to model specific market conditions.
4. **Using price control parameters** to influence price trajectories.
5. **Generating and interpreting comparative charts** across scenarios.

These techniques allow for exploration of market conditions under varied market dynamics and policy settings when using NZUpy.